<a href="https://colab.research.google.com/github/anmolpandeybtech/Skin_Disease_Hackathon/blob/main/Skin_Disease_resnet152V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from IPython.display import Javascript

def disable_colab_idle_timeout():
    js_code = '''
    function ClickConnect(){
        console.log("Working");
        document.querySelector("colab-toolbar-button#connect").click()
    }
    setInterval(ClickConnect,60000)
    '''
    display(Javascript(js_code))

disable_colab_idle_timeout()

<IPython.core.display.Javascript object>

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [3]:
!kaggle datasets download -d ascanipek/skin-diseases

100% 6.18G/6.18G [01:06<00:00, 117MB/s]
100% 6.18G/6.18G [01:06<00:00, 100MB/s]


In [4]:
import zipfile
zip_ref = zipfile.ZipFile('/content/skin-diseases.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [5]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob

In [6]:
IMAGE_SIZE = [224, 224]

train_path = '/content/kaggle/train'
valid_path = '/content/kaggle/val'

In [7]:
import tensorflow
resnet152V2 =tensorflow.keras.applications.ResNet152V2(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

234545216/234545216 [==============================] - 1s 0us/step


In [8]:
for layer in resnet152V2.layers:
    layer.trainable = False

In [11]:
folders = glob('/content/kaggle/train/*')

In [12]:
x = Flatten()(resnet152V2.output)

In [13]:
prediction = Dense(len(folders), activation='softmax')(x)

model = Model(inputs=resnet152V2.input, outputs=prediction)

In [14]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 pool1_pad (ZeroPadding2D)   (None, 114, 114, 64)         0         ['conv1_conv[0][0]']          
                                                                                              

In [15]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [16]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [18]:
training_set = train_datagen.flow_from_directory('/content/kaggle/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 30909 images belonging to 6 classes.


In [19]:
validation_set = train_datagen.flow_from_directory('/content/kaggle/val',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 3923 images belonging to 6 classes.


In [21]:
test_set = test_datagen.flow_from_directory('/content/kaggle/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 3928 images belonging to 6 classes.


In [4]:
r = model.fit_generator(
  training_set,
  validation_data=validation_set,
  epochs=20,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

NameError: ignored

In [3]:

# Save the model to a file
model.save("my_model.h5")

NameError: ignored